# **Sweeps**

## **Pre-Sweep**

### **Import Dependencies**

In [1]:
import wandb
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

### **Define Dataset Class**

In [2]:
class TimeSeriesDataset(Dataset):
    def __init__(self, time_series_list, burst_ids):
        self.time_series_list = time_series_list
        self.burst_ids = burst_ids

    def __len__(self):
        return len(self.time_series_list)

    def __getitem__(self, idx):
        return self.time_series_list[idx], self.burst_ids[idx]

### **Define Model Components**

In [3]:
# Define the Positional Encoding Class
class PositionalEncoding(nn.Module):
    def __init__(self, model_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, model_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_dim, 2).float() * (-math.log(10000.0) / model_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

# Define the Channel Embedding Class
class ChannelEmbedding(nn.Module):
    def __init__(self, input_dim, model_dim):
        super(ChannelEmbedding, self).__init__()
        self.channel_embedding = nn.Linear(input_dim, model_dim)

    def forward(self, x):
        channel_indices = torch.arange(input_dim).unsqueeze(0).expand(x.size(0), x.size(1), -1) 
        channel_embedded = self.channel_embedding(channel_indices.float())   
        return x + channel_embedded    

# Define the Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.embedding = nn.Sequential(
            nn.Linear(input_dim, model_dim),
            nn.ReLU()  
        )
        self.channel_embedding = ChannelEmbedding(input_dim, model_dim)
        self.positional_encoding = PositionalEncoding(model_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, src):
        src = self.embedding(src) 
        src = self.channel_embedding(src)        
        src = self.positional_encoding(src)
        output = self.transformer_encoder(src)
        return output    

# Define the Transformer Decoder
class TransformerDecoder(nn.Module):
    def __init__(self, model_dim, output_dim, num_heads, num_layers, dropout=0.1):
        super(TransformerDecoder, self).__init__()
        decoder_layer = nn.TransformerDecoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.output_layer = nn.Linear(model_dim, output_dim)

    def forward(self, tgt, memory):
        output = self.transformer_decoder(tgt, memory)
        output = self.output_layer(output)
        return output

# Define the Compression Neural Network
class IntermediateNetwork(nn.Module):
    def __init__(self, latent_dim, reduced_dim, sequence_length):
        super(IntermediateNetwork, self).__init__()
        self.reduce = nn.Sequential(
            nn.Linear(sequence_length * latent_dim, reduced_dim),
            nn.ReLU()  
        )
        self.expand = nn.Sequential(
            nn.Linear(reduced_dim, sequence_length * latent_dim),
            nn.ReLU() 
        )

    def forward(self, x):
        # Reduce dimensionality
        org_shape = x.size()
        x = x.reshape(x.size(1), -1)  # Reshape to (batch_size, sequence_length * latent_dim)
        reduced_x = self.reduce(x)

        # Expand dimensionality
        expanded_x = self.expand(reduced_x)
        expanded_x = expanded_x.reshape(org_shape)  # Reshape back

        return expanded_x, reduced_x 
    
# Define the Autoencoder
class TransformerAutoencoder(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, sequence_length, reduced_dim, dropout=0.1):  
        super(TransformerAutoencoder, self).__init__()
        self.encoder = TransformerEncoder(input_dim, model_dim, num_heads, num_layers, dropout)
        self.intermediate_network = IntermediateNetwork(model_dim, reduced_dim, sequence_length)
        self.decoder = TransformerDecoder(model_dim, input_dim, num_heads, num_layers, dropout)  

    def forward(self, src):
        memory = self.encoder(src)
        expanded_latent, reduced_latent = self.intermediate_network(memory)  
        tgt = self.encoder.embedding(src)  
        tgt = self.encoder.positional_encoding(tgt)  
        output = self.decoder(tgt, expanded_latent)  
        return output, reduced_latent  

### **Data Loading**

In [4]:
def load_data():
    lcs = pd.read_csv('lcs.csv')
    channels = ['n0','n1','n2','n3','n4','n5','n6','n7','n8','n9','na','nb','b0','b1']

    for ch in channels:
        missing = lcs[ch].isnull()
        lcs.loc[missing, ch] = np.random.normal(lcs[ch].mean(), lcs[ch].std(), missing.sum())

    time_series_list, burst_ids = [], []
    for burst, group in lcs.groupby('burst'):
        data = group[channels].values
        time_series_list.append(torch.tensor(data, dtype=torch.float32))
        burst_ids.append(burst)

    time_series_list = nn.utils.rnn.pad_sequence(time_series_list, batch_first=True, padding_value=0.0)
    scaler = StandardScaler()
    flat = time_series_list.reshape(time_series_list.shape[0], -1)
    flat = scaler.fit_transform(flat)
    time_series_list = flat.reshape(time_series_list.shape)
    return time_series_list, burst_ids

## **Sweep**

### **Sweep Training Function**

In [5]:
def train_autoencoder_sweep():
    wandb.init()
    config = wandb.config

    # Load data
    time_series_list, burst_ids = load_data()
    dataset = TimeSeriesDataset(time_series_list, burst_ids)
    dataloader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True)

    seq_len = time_series_list.shape[1]
    model = TransformerAutoencoder(
        input_dim=14,
        model_dim=config.model_dim,
        num_heads=config.num_heads,
        num_layers=config.num_layers,
        sequence_length=seq_len,
        reduced_dim=config.reduced_dim,
        dropout=config.dropout
    )

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    for epoch in range(config.num_epochs):
        for batch, _ in dataloader:
            batch = batch.permute(1, 0, 2).float()
            reconstructed, _ = model(batch)
            loss = criterion(reconstructed, batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        wandb.log({
            "epoch": epoch,
            "loss": loss.item(),
            "learning_rate": optimizer.param_groups[0]['lr']
        })

    wandb.log({"final_loss": loss.item()})
    wandb.finish()

### **Sweep Config** - _THE IMPORTANT PART_

In [6]:
num_sweeps = 20

# Flags to toggle which hyperparameters to sweep
config_flags = {
    'sweep_model_dim': False,
    'sweep_num_heads': False,
    'sweep_num_layers': False,
    'sweep_reduced_dim': False,
    'sweep_batch_size': False,
    'sweep_learning_rate': True,
    'sweep_dropout': False,
    'sweep_method': 'bayes'  # 'random', 'bayes', or 'grid'
}

# Default values if not swept
fixed_defaults = {
    'input_dim': 14,
    'model_dim': 32,
    'num_heads': 4,
    'num_layers': 2,
    'reduced_dim': 1024,
    'batch_size': 16,
    'learning_rate': 0.0001,
    'dropout': 0.1,
    'num_epochs': 20
    # 'sequence_length' will be set dynamically from data
}

def generate_sweep_config(flags, defaults):
    sweep_config = {
        'method': flags['sweep_method'],
        'metric': {'name': 'loss', 'goal': 'minimize'},
        'parameters': {}
    }

    if flags['sweep_model_dim']:
        sweep_config['parameters']['model_dim'] = {'values': [32, 64, 128]}
    else:
        sweep_config['parameters']['model_dim'] = {'value': defaults['model_dim']}

    if flags['sweep_num_heads']:
        sweep_config['parameters']['num_heads'] = {'values': [2, 4, 8]}
    else:
        sweep_config['parameters']['num_heads'] = {'value': defaults['num_heads']}

    if flags['sweep_num_layers']:
        sweep_config['parameters']['num_layers'] = {'values': [1, 2, 3]}
    else:
        sweep_config['parameters']['num_layers'] = {'value': defaults['num_layers']}

    if flags['sweep_reduced_dim']:
        sweep_config['parameters']['reduced_dim'] = {'values': [256, 512, 1024]}
    else:
        sweep_config['parameters']['reduced_dim'] = {'value': defaults['reduced_dim']}

    if flags['sweep_batch_size']:
        sweep_config['parameters']['batch_size'] = {'values': [8, 16, 32]}
    else:
        sweep_config['parameters']['batch_size'] = {'value': defaults['batch_size']}

    if flags['sweep_learning_rate']:
        sweep_config['parameters']['learning_rate'] = {
            'distribution': 'log_uniform_values',
            'min': 1e-5,
            'max': 1e-2
        }
    else:
        sweep_config['parameters']['learning_rate'] = {'value': defaults['learning_rate']}

    if flags['sweep_dropout']:
        sweep_config['parameters']['dropout'] = {'values': [0.0, 0.1, 0.2]}
    else:
        sweep_config['parameters']['dropout'] = {'value': defaults['dropout']}

    sweep_config['parameters']['num_epochs'] = {'value': defaults['num_epochs']}

    return sweep_config

### **Run Sweep**

In [ ]:

sweep_config = generate_sweep_config(config_flags, fixed_defaults)
sweep_id = wandb.sweep(sweep_config, project="GBM-LSTM-Sweep")
wandb.agent(sweep_id, function=train_autoencoder_sweep, count=num_sweeps)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: lf1kv01s
Sweep URL: https://wandb.ai/tobiassafie-drexel-university/GBM-LSTM-Sweep/sweeps/lf1kv01s
 lf1kv01s
Sweep URL: https://wandb.ai/tobiassafie-drexel-university/GBM-LSTM-Sweep/sweeps/lf1kv01s


wandb: Agent Starting Run: gbuyuxsm with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.1
wandb: 	learning_rate: 0.00193007480382725
wandb: 	model_dim: 32
wandb: 	num_epochs: 20
wandb: 	num_heads: 4
wandb: 	num_layers: 2
wandb: 	reduced_dim: 1024
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: 	batch_size: 16
wandb: 	dropout: 0.1
wandb: 	learning_rate: 0.00193007480382725
wandb: 	model_dim: 32
wandb: 	num_epochs: 20
wandb: 	num_heads: 4
wandb: 	num_layers: 2
wandb: 	reduced_dim: 1024
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tobiassafie (tobiassafie-drexel-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: tobiassafie (tobiassafie-drexel-university) to https://api.wandb.